In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
import torch.optim as optim

In [2]:
# IMU-Based Energy Expenditure Estimation for Various Walking Conditions Using a Hybrid CNN–LSTM Model
# conv1d(64, 3)
# maxpooling1d(64, 3)
# 2 LSTM 64 hidden units
# flatten layer
# fully connected layer
# output

In [3]:
class CNN_LSTM(nn.Module):
    def __init__(self, num_data, out_channels=64, kernel_conv=1, kernel_pool=1, hidden_lstm=64, layers_lstm=1):
        super(CNN_LSTM, self).__init__()
        self.batch_size = batch_size

        # Linear layer
        self.linear1 = nn.Linear(time_steps*num_data, num_data)

        # 1D Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=num_data, out_channels=64, kernel_size=kernel_conv).double()

        # Max Pooling Layer
        self.pool1 = nn.MaxPool1d(kernel_size=kernel_pool).double()

        # LSTM layers
        self.lstm = nn.LSTM(out_channels, hidden_lstm, layers_lstm, batch_first=True).double()

        # Fully connected layer for regression
        self.fc1 = nn.Linear(out_channels, 1).double() # Output dimension is 1 for regression

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 1e-1)
                nn.init.zeros_(m.bias)

    def forward(self, x): # shape (batch_size, 1, time_steps, num_data) = (2, 1, 100, 15)
        x = x.view(batch_size, -1) # shape (batch_size, 1*time_steps*num_data) = (2, 150)
        x = self.linear1(x.float()) # shape (batch_size, 1*num_data) = (2, 15)
        x = x.view(batch_size, 1, 15) # shape (batch_size, 1, 1, num_data) = (2, 1, 15)
        x = F.relu(x) # shape (batch_size, num_sensors, 1, num_data) = (2, 1, 15)

        x = x.double()
        x = x.permute(0, 2, 1) # shape (batch_size, num_data, num_sensors) = (2, 15, 1)
        x = self.conv1(x) # shape (batch_size, out_channels, _) = (2, 64, 1)
        x = self.pool1(x) # shape (batch_size, out_channels, 1) = (2, 64, 1)
        x = F.relu(x) # shape (batch_size, out_channels, 1) = (2, 64, 1)

        x = x.squeeze(2) # shape (batch_size, out_channels) = (2, 64)
        x, _ = self.lstm(x) # shape (batch_size, out_channels) = (2, 64)

        x = self.fc1(x).float() # shape (batch_size, 1) = (2, 1)
        output = x.squeeze(1) # shape (batch_size) = (2)
        return output

In [4]:
import sys
def train(model, train_loader, time_steps, criterion, optimizer, epochs=5, batch_size=1):
    model.train()
    losses = []
    for epoch in range(epochs):
        running_loss = 0.0
        print('-----------EPOCH %d-----------' % (epoch + 1))
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.float()
            labels = labels.float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 250 == 0:
                print('[episode: %d] loss: %f' % (i + 1, running_loss / 250))
                losses.append(running_loss / 250)
                running_loss = 0.0
    print(outputs, labels)
    plt.plot(losses)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.show()

def test(model, test_loader, batch_size=1):
    model.eval()
    total_error = 0
    total_batches = 0
    total_error2 = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data
            outputs = model(inputs)
            total_batches += 1
            batch_error = torch.abs(outputs.flatten() - labels.flatten()) / labels.flatten()
            batch_error2 = torch.abs(outputs.flatten() - labels.flatten())
            total_error += batch_error.mean().item()
            total_error2 += batch_error2.mean().item()
            if (i + 1) % 250 == 0:
                print(outputs, labels)
    print('Average accuracy: %f %% ' % (100 - total_error / total_batches * 100))
    print('Average error between prediction and label: %f' % (total_error2 / total_batches))

In [10]:
# class SimpleDataset:
def SimpleDataset(person, weight, attempt, time_steps, num_data):
    all_data = []

    for p in range(1, person + 1):
        for w in range(1, weight + 1):
            for a in range(1, attempt + 1):

                imu_data = torch.tensor((w+1)*10*np.random.rand(1, time_steps, num_data))
                emg_data = torch.mean(imu_data) / 40
                if emg_data > 1:
                  emg_data = 1

                all_data.append((imu_data, emg_data))
        # print(f'Person {p}/{person} done')

    # for p in range(1, self.person + 1):
    #     for w in range(1, self.weight + 1):
    #         for a in range(1, self.attempt + 1):
    #             imu_data = self.load_imu_data(p, w, a)
    #             emg_data = self.load_emg_data(p, w, a)

    #             # Process EMG data
    #             max_each_sensor = torch.max(emg_data, axis=0).values  # Find maximum value in each column
    #             max_tot = torch.max(max_each_sensor)  # Find maximum value across all columns
    #             label_emg = max_tot / self.mvc

    #             all_data.append((imu_data, label_emg))

    #     print(f'Person {p}/{self.person} done')

    return all_data

# def __init__(self, num_moves, num_sensors, num_diff_acts, num_data, time_steps=100, batch_size=1):

#     self.data = torch.zeros((num_moves, num_sensors, time_steps, num_data))
#     for i in range(num_diff_acts):
#         for j in range(np.int64(num_moves/num_diff_acts)):
#             self.data[(i+j+(np.int64(num_moves/num_diff_acts)-1)*i), :, :, :] = torch.tensor((i+1)*10*np.random.rand(1, num_sensors, time_steps, num_data))

#     self.labels = torch.zeros((num_moves, 1, 1))
#     for i in range(num_moves):
#         self.labels[i, :, :] = torch.mean(self.data[i, :, :, :]) / 60

# def __len__(self):
#     return self.num_samples

# def __getitem__(self, indices):
#     idx_1, idx_2 = indices
#     return self.data[idx_1, :, idx_2, :], self.labels[idx_1, :, :].numpy()

a = np.int64(10*np.random.rand() + 3)
num_person = 1000
num_weights = 5
num_attempt = 50
# num_moves = 1000
time_steps = 100
# num_sensors = 1
num_data = 15
# dataset = SimpleDataset(num_moves, num_sensors, num_weights, num_data, time_steps)
dataset = SimpleDataset(num_person, num_weights, num_attempt, time_steps, num_data)

In [6]:
data, label = dataset[14]
print(np.shape(data)[1])
# print(dataset[:, :])

100


In [11]:
# data, labels = dataset[:, :]
# num_rows = data.shape[0]
# indices = torch.randperm(num_rows)
# shuffled_data = data[indices]
# shuffled_labels = labels[indices]
# shuffled_dataset = shuffled_data, shuffled_labels

batch_size = 10

# train_set = [(shuffled_data[i], shuffled_labels[i]) for i in range(np.int64(num_moves/2))]
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

# test_set = [(shuffled_data[i], shuffled_labels[i]) for i in range(np.int64(num_moves/2), num_moves)]
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

train_set = [(dataset[i]) for i in range(np.int64(len(dataset)/2))]
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = [(dataset[i]) for i in range(np.int64(len(dataset)/2), len(dataset))]
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)


In [12]:
model = CNN_LSTM(num_data, layers_lstm=2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train(model, train_loader, time_steps, criterion, optimizer, epochs=3, batch_size=batch_size)

-----------EPOCH 1-----------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[episode: 250] loss: 0.042670
[episode: 500] loss: 0.032854
[episode: 750] loss: 0.031515
[episode: 1000] loss: 0.031678
[episode: 1250] loss: 0.031623
[episode: 1500] loss: 0.030583
[episode: 1750] loss: 0.029894
[episode: 2000] loss: 0.029191
[episode: 2250] loss: 0.029572
[episode: 2500] loss: 0.029719
[episode: 2750] loss: 0.027909
[episode: 3000] loss: 0.028528
[episode: 3250] loss: 0.026912
[episode: 3500] loss: 0.025821
[episode: 3750] loss: 0.025359
[episode: 4000] loss: 0.025564
[episode: 4250] loss: 0.023053
[episode: 4500] loss: 0.022377
[episode: 4750] loss: 0.020595
[episode: 5000] loss: 0.019746
[episode: 5250] loss: 0.018101
[episode: 5500] loss: 0.016512
[episode: 5750] loss: 0.014932
[episode: 6000] loss: 0.013284
[episode: 6250] loss: 0.012129
[episode: 6500] loss: 0.011318
[episode: 6750] loss: 0.010811
[episode: 7000] loss: 0.009357
[episode: 7250] loss: 0.009410
[episode: 7500] loss: 0.008956
[episode: 7750] loss: 0.008397
[episode: 8000] loss: 0.007780
[episode: 8

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[episode: 12500] loss: 0.003907
-----------EPOCH 2-----------
[episode: 250] loss: 0.003464
[episode: 500] loss: 0.003134
[episode: 750] loss: 0.003459
[episode: 1000] loss: 0.003381
[episode: 1250] loss: 0.003116
[episode: 1500] loss: 0.003211
[episode: 1750] loss: 0.003163
[episode: 2000] loss: 0.003125
[episode: 2250] loss: 0.003167
[episode: 2500] loss: 0.002606
[episode: 2750] loss: 0.002564
[episode: 3000] loss: 0.002571
[episode: 3250] loss: 0.002838
[episode: 3500] loss: 0.002455
[episode: 3750] loss: 0.002448
[episode: 4000] loss: 0.002522
[episode: 4250] loss: 0.002435
[episode: 4500] loss: 0.002351
[episode: 4750] loss: 0.002289
[episode: 5000] loss: 0.002244
[episode: 5250] loss: 0.002358
[episode: 5500] loss: 0.002176
[episode: 5750] loss: 0.002428
[episode: 6000] loss: 0.002389
[episode: 6250] loss: 0.002289
[episode: 6500] loss: 0.001871
[episode: 6750] loss: 0.002166
[episode: 7000] loss: 0.002017
[episode: 7250] loss: 0.001995
[episode: 7500] loss: 0.002026
[episode: 7

In [13]:
test(model, test_loader, batch_size=batch_size)

tensor([0.4342, 0.3930, 0.2633, 0.6581, 0.5676, 0.4997, 0.6763, 0.4166, 0.2616,
        0.6835]) tensor([0.3649, 0.3774, 0.2579, 0.6354, 0.5039, 0.4868, 0.6394, 0.3732, 0.2556,
        0.6386], dtype=torch.float64)
tensor([0.6448, 0.4399, 0.6547, 0.2422, 0.7455, 0.6200, 0.2461, 0.3842, 0.3767,
        0.3913]) tensor([0.6214, 0.3697, 0.6238, 0.2490, 0.7440, 0.6090, 0.2441, 0.3839, 0.3686,
        0.3667], dtype=torch.float64)
tensor([0.4465, 0.3758, 0.4344, 0.2617, 0.6022, 0.2371, 0.7737, 0.5260, 0.6617,
        0.3795]) tensor([0.3749, 0.3798, 0.3754, 0.2467, 0.5076, 0.2502, 0.7461, 0.5139, 0.6275,
        0.3722], dtype=torch.float64)
tensor([0.2966, 0.5402, 0.7718, 0.4043, 0.5620, 0.3769, 0.7642, 0.7884, 0.7334,
        0.5451]) tensor([0.2529, 0.5024, 0.7223, 0.3803, 0.5036, 0.3633, 0.7563, 0.7494, 0.7583,
        0.4960], dtype=torch.float64)
tensor([0.2949, 0.6527, 0.6507, 0.2633, 0.5008, 0.5912, 0.4038, 0.2622, 0.5283,
        0.6589]) tensor([0.2461, 0.6210, 0.6241, 0.2483, 0.4